Connected to Python 3.12.0

In [1]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])

<ipython-input-1-0ea7f5060e54>:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


def accumulate(iterable, func=operator.add):
    'Return running totals'
    # accumulate([1,2,3,4,5]) --> 1 3 6 10 15
    # accumulate([1,2,3,4,5], operator.mul) --> 1 2 6 24 120
    it = iter(iterable)
    total = next(it)
    yield total
    for element in it:
        total = func(total, element)
        yield total

def find_lt(a, x):
    """ Find rightmost value less than x"""
    i = bisect_left(a, x)
    if i:
        return int(i-1)
    print('in find_lt,{}'.format(a))
    raise ValueError


def log_gampoiss(k,alpha,beta):
    import math
    k = int(k)
    return math.lgamma(k+alpha)+alpha*np.log(beta)-math.lgamma(alpha)-math.lgamma(k+1)-(alpha+k)*np.log(1+beta)


def log_betabin(k,n,alpha,beta):
    import math
    try:
        Const =  math.lgamma(alpha + beta) - math.lgamma(alpha) - math.lgamma(beta)
    except:
        print('alpha = {}, beta = {}'.format(alpha,beta))
    if isinstance(k,list) or isinstance(k,np.ndarray):
        if len(k)!=len(n):
            print('length of k is %d and length of n is %d'%(len(k),len(n)))
            raise ValueError
        lbeta = []
        for ki,ni in zip(k,n):
            # lbeta.append(math.lgamma(ni+1)- math.lgamma(ki+1) - math.lgamma(ni-ki+1) + math.lgamma(ki+alpha) + math.lgamma(ni-ki+beta) - math.lgamma(ni+alpha+beta) + Const)
            lbeta.append(math.lgamma(ki+alpha) + math.lgamma(ni-ki+beta) - math.lgamma(ni+alpha+beta) + Const)
        return np.array(lbeta)
    else:
        return math.lgamma(k+alpha) + math.lgamma(n-k+beta) - math.lgamma(n+alpha+beta) + Const
        # return math.lgamma(n+1)- math.lgamma(k+1) - math.lgamma(n-k+1) + math.lgamma(k+alpha) + math.lgamma(n-k+beta) - math.lgamma(n+alpha+beta) + Const

def getConfusion(Yhat,Y):
    if len(Yhat)!=len(Y):
        raise NameError('Yhat has different length')
    TP = np.dot(np.array(Y),np.array(Yhat))
    FP = np.sum(Yhat) - TP
    TN = len(Y) - np.sum(Y)-FP
    FN = len(Yhat) - np.sum(Yhat) - TN
    return TP,FP,TN,FN

def predict(rules,df):
    Z = [[] for rule in rules]
    dfn = 1-df #df has negative associations
    dfn.columns = [name.strip() + '_neg' for name in df.columns]
    df = pd.concat([df,dfn],axis = 1)
    for i,rule in enumerate(rules):
        Z[i] = (np.sum(df[list(rule)],axis=1)==len(rule)).astype(int)
    Yhat = (np.sum(Z,axis=0)>0).astype(int)
    return Yhat

def extract_rules(tree, feature_names):
    left      = tree.tree_.children_left
    right     = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]
    # get ids of child nodes
    idx = np.argwhere(left == -1)[:,0]     

    def recurse(left, right, child, lineage=None):          
        if lineage is None:
            lineage = []
        if child in left:
            parent = np.where(left == child)[0].item()
            suffix = '_neg'
        else:
            parent = np.where(right == child)[0].item()
            suffix = ''

        #           lineage.append((parent, split, threshold[parent], features[parent]))
        lineage.append((features[parent].strip()+suffix))

        if parent == 0:
            lineage.reverse()
            return lineage
        else:
            return recurse(left, right, parent, lineage)   
    rules = []
    for child in idx:
        rule = []
        for node in recurse(left, right, child):
            rule.append(node)
        rules.append(rule)
    return rules


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random.random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random.random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random.random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random.random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random.random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])



""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.591s to generate 19772 rules
Screening rules using information gain


<ipython-input-2-958436519346>:206: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-2-958436519346>:206: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-2-958436519346>:208: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-2-958436519346>:208: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-2-958436519346>:265: RuntimeWarning: overflow encountered in exp
  alpha = np.exp(float(pt_new -pt_curr)/T)


	Took 0.548s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!
found a more accurate model: accuracy = 0.5417910447761194

** chain = 0, max at iter = 0 ** 
 accuracy = 0.5417910447761194, TP = 249.0,FP = 117.0, TN = 114.0, FN = 190.0
 pt_new is -832.27859463412, prior_ChsRules=-55.76454119002847, likelihood_1 = -340.0904328557817, likelihood_2 = -436.42362058830986
 
['5_X', '1_B', '9_O_neg']
['5_X_neg', '5_O', '1_O']
['9_O_neg', '5_O', '3_X_neg']
['4_B_neg', '5_X_neg', '9_O']
['3_O_neg', '5_O_neg', '7_O_neg']
[1852, 839, 65, 33, 1958]
found a more accurate model: accuracy = 0.6686567164179105

** chain = 0, max at iter = 1 ** 
 accuracy = 0.6686567164179105, TP = 334.0,FP = 117.0, TN = 114.0, FN = 105.0
 pt_new is -714.3872831851045, prior_ChsRules=-66.49072063146559, likelihood_1 = -351.919381133579, likelihood_2 = -295.9771814200599
 
['5_X', '1_B', '9_O_

NameError: name 'bisect_left' is not defined

In [3]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


def accumulate(iterable, func=operator.add):
    'Return running totals'
    # accumulate([1,2,3,4,5]) --> 1 3 6 10 15
    # accumulate([1,2,3,4,5], operator.mul) --> 1 2 6 24 120
    it = iter(iterable)
    total = next(it)
    yield total
    for element in it:
        total = func(total, element)
        yield total

def find_lt(a, x):
    """ Find rightmost value less than x"""
    i = bisect_left(a, x)
    if i:
        return int(i-1)
    print('in find_lt,{}'.format(a))
    raise ValueError


def log_gampoiss(k,alpha,beta):
    import math
    k = int(k)
    return math.lgamma(k+alpha)+alpha*np.log(beta)-math.lgamma(alpha)-math.lgamma(k+1)-(alpha+k)*np.log(1+beta)


def log_betabin(k,n,alpha,beta):
    import math
    try:
        Const =  math.lgamma(alpha + beta) - math.lgamma(alpha) - math.lgamma(beta)
    except:
        print('alpha = {}, beta = {}'.format(alpha,beta))
    if isinstance(k,list) or isinstance(k,np.ndarray):
        if len(k)!=len(n):
            print('length of k is %d and length of n is %d'%(len(k),len(n)))
            raise ValueError
        lbeta = []
        for ki,ni in zip(k,n):
            # lbeta.append(math.lgamma(ni+1)- math.lgamma(ki+1) - math.lgamma(ni-ki+1) + math.lgamma(ki+alpha) + math.lgamma(ni-ki+beta) - math.lgamma(ni+alpha+beta) + Const)
            lbeta.append(math.lgamma(ki+alpha) + math.lgamma(ni-ki+beta) - math.lgamma(ni+alpha+beta) + Const)
        return np.array(lbeta)
    else:
        return math.lgamma(k+alpha) + math.lgamma(n-k+beta) - math.lgamma(n+alpha+beta) + Const
        # return math.lgamma(n+1)- math.lgamma(k+1) - math.lgamma(n-k+1) + math.lgamma(k+alpha) + math.lgamma(n-k+beta) - math.lgamma(n+alpha+beta) + Const

def getConfusion(Yhat,Y):
    if len(Yhat)!=len(Y):
        raise NameError('Yhat has different length')
    TP = np.dot(np.array(Y),np.array(Yhat))
    FP = np.sum(Yhat) - TP
    TN = len(Y) - np.sum(Y)-FP
    FN = len(Yhat) - np.sum(Yhat) - TN
    return TP,FP,TN,FN

def predict(rules,df):
    Z = [[] for rule in rules]
    dfn = 1-df #df has negative associations
    dfn.columns = [name.strip() + '_neg' for name in df.columns]
    df = pd.concat([df,dfn],axis = 1)
    for i,rule in enumerate(rules):
        Z[i] = (np.sum(df[list(rule)],axis=1)==len(rule)).astype(int)
    Yhat = (np.sum(Z,axis=0)>0).astype(int)
    return Yhat

def extract_rules(tree, feature_names):
    left      = tree.tree_.children_left
    right     = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]
    # get ids of child nodes
    idx = np.argwhere(left == -1)[:,0]     

    def recurse(left, right, child, lineage=None):          
        if lineage is None:
            lineage = []
        if child in left:
            parent = np.where(left == child)[0].item()
            suffix = '_neg'
        else:
            parent = np.where(right == child)[0].item()
            suffix = ''

        #           lineage.append((parent, split, threshold[parent], features[parent]))
        lineage.append((features[parent].strip()+suffix))

        if parent == 0:
            lineage.reverse()
            return lineage
        else:
            return recurse(left, right, parent, lineage)   
    rules = []
    for child in idx:
        rule = []
        for node in recurse(left, right, child):
            rule.append(node)
        rules.append(rule)
    return rules


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random.random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random.random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random.random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random.random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random.random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])



""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.722s to generate 19128 rules
Screening rules using information gain
	Took 0.493s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!
found a more accurate model: accuracy = 0.6268656716417911

** chain = 0, max at iter = 0 ** 
 accuracy = 0.6268656716417911, TP = 216.0,FP = 27.0, TN = 204.0, FN = 223.0
 pt_new is -642.5921753983212, prior_ChsRules=-20.473457780984972, likelihood_1 = -113.81219883246558, likelihood_2 = -508.3065187848706
 
['5_O_neg', '6_O']
['9_O_neg', '1_O_neg', '5_O_neg']
[502, 1833]
found a more accurate model: accuracy = 0.7149253731343284

** chain = 0, max at iter = 1 ** 
 accuracy = 0.7149253731343284, TP = 275.0,FP = 27.0, TN = 204.0, FN = 164.0
 pt_new is -565.1817153618795, prior_ChsRules=-31.50979800647292, likelihood_1 = -115.99009123329961, likelihood_2 = -417.68182612210694
 
['5_O_neg', '6_O']
['9_O_neg', '1_O_neg', '5_O_

<ipython-input-3-958436519346>:206: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-3-958436519346>:206: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-3-958436519346>:208: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-3-958436519346>:208: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-3-958436519346>:265: RuntimeWarning: overflow encountered in exp
  alpha = np.exp(float(pt_new -pt_curr)/T)


NameError: name 'bisect_left' is not defined

In [4]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


def accumulate(iterable, func=operator.add):
    'Return running totals'
    # accumulate([1,2,3,4,5]) --> 1 3 6 10 15
    # accumulate([1,2,3,4,5], operator.mul) --> 1 2 6 24 120
    it = iter(iterable)
    total = next(it)
    yield total
    for element in it:
        total = func(total, element)
        yield total

def find_lt(a, x):
    """ Find rightmost value less than x"""
    i = bisect_left(a, x)
    if i:
        return int(i-1)
    print('in find_lt,{}'.format(a))
    raise ValueError


def log_gampoiss(k,alpha,beta):
    import math
    k = int(k)
    return math.lgamma(k+alpha)+alpha*np.log(beta)-math.lgamma(alpha)-math.lgamma(k+1)-(alpha+k)*np.log(1+beta)


def log_betabin(k,n,alpha,beta):
    import math
    try:
        Const =  math.lgamma(alpha + beta) - math.lgamma(alpha) - math.lgamma(beta)
    except:
        print('alpha = {}, beta = {}'.format(alpha,beta))
    if isinstance(k,list) or isinstance(k,np.ndarray):
        if len(k)!=len(n):
            print('length of k is %d and length of n is %d'%(len(k),len(n)))
            raise ValueError
        lbeta = []
        for ki,ni in zip(k,n):
            # lbeta.append(math.lgamma(ni+1)- math.lgamma(ki+1) - math.lgamma(ni-ki+1) + math.lgamma(ki+alpha) + math.lgamma(ni-ki+beta) - math.lgamma(ni+alpha+beta) + Const)
            lbeta.append(math.lgamma(ki+alpha) + math.lgamma(ni-ki+beta) - math.lgamma(ni+alpha+beta) + Const)
        return np.array(lbeta)
    else:
        return math.lgamma(k+alpha) + math.lgamma(n-k+beta) - math.lgamma(n+alpha+beta) + Const
        # return math.lgamma(n+1)- math.lgamma(k+1) - math.lgamma(n-k+1) + math.lgamma(k+alpha) + math.lgamma(n-k+beta) - math.lgamma(n+alpha+beta) + Const

def getConfusion(Yhat,Y):
    if len(Yhat)!=len(Y):
        raise NameError('Yhat has different length')
    TP = np.dot(np.array(Y),np.array(Yhat))
    FP = np.sum(Yhat) - TP
    TN = len(Y) - np.sum(Y)-FP
    FN = len(Yhat) - np.sum(Yhat) - TN
    return TP,FP,TN,FN

def predict(rules,df):
    Z = [[] for rule in rules]
    dfn = 1-df #df has negative associations
    dfn.columns = [name.strip() + '_neg' for name in df.columns]
    df = pd.concat([df,dfn],axis = 1)
    for i,rule in enumerate(rules):
        Z[i] = (np.sum(df[list(rule)],axis=1)==len(rule)).astype(int)
    Yhat = (np.sum(Z,axis=0)>0).astype(int)
    return Yhat

def extract_rules(tree, feature_names):
    left      = tree.tree_.children_left
    right     = tree.tree_.children_right
    threshold = tree.tree_.threshold
    features  = [feature_names[i] for i in tree.tree_.feature]
    # get ids of child nodes
    idx = np.argwhere(left == -1)[:,0]     

    def recurse(left, right, child, lineage=None):          
        if lineage is None:
            lineage = []
        if child in left:
            parent = np.where(left == child)[0].item()
            suffix = '_neg'
        else:
            parent = np.where(right == child)[0].item()
            suffix = ''

        #           lineage.append((parent, split, threshold[parent], features[parent]))
        lineage.append((features[parent].strip()+suffix))

        if parent == 0:
            lineage.reverse()
            return lineage
        else:
            return recurse(left, right, parent, lineage)   
    rules = []
    for child in idx:
        rule = []
        for node in recurse(left, right, child):
            rule.append(node)
        rules.append(rule)
    return rules


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random.random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random.random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random.random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random.random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random.random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])



""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.613s to generate 18984 rules
Screening rules using information gain
	Took 0.496s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.002s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!
found a more accurate model: accuracy = 0.6582089552238806

** chain = 0, max at iter = 0 ** 
 accuracy = 0.6582089552238806, TP = 333.0,FP = 124.0, TN = 108.0, FN = 105.0
 pt_new is -717.0230701141477, prior_ChsRules=-55.76454119002847, likelihood_1 = -366.24195365895866, likelihood_2 = -295.01657526516055
 
['5_X_neg', '5_B_neg', '1_O_neg']
['5_X', '4_B_neg', '7_O_neg']
['3_O_neg', '2_X_neg', '5_X']
['9_O_neg', '8_O', '5_X']
['5_O_neg', '9_O_neg', '1_O_neg']
[162, 778, 1541, 501, 1997]
found a more accurate model: accuracy = 0.7059701492537314

** chain = 0, max at iter = 1 ** 
 accuracy = 0.7059701492537314, TP = 365.0,FP = 124.0, TN = 108.0, FN = 73.0
 pt_new is -666.2738164123116, prior_ChsRules=-66.49072063146559, li

<ipython-input-4-958436519346>:206: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-4-958436519346>:206: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-4-958436519346>:208: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-4-958436519346>:208: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-4-958436519346>:265: RuntimeWarning: overflow encountered in exp
  alpha = np.exp(float(pt_new -pt_curr)/T)


NameError: name 'bisect_left' is not defined

In [5]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])

In [6]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])

In [7]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])
            

""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.659s to generate 18608 rules
Screening rules using information gain
	Took 0.508s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!


<ipython-input-7-7b66c4721dab>:107: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-7-7b66c4721dab>:107: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-7-7b66c4721dab>:109: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-7-7b66c4721dab>:109: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]


TypeError: 'module' object is not callable. Did you mean: 'random.random(...)'?

In [8]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])
            

""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.655s to generate 18594 rules
Screening rules using information gain
	Took 0.502s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!


<ipython-input-8-1d36f7cd83a1>:107: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-8-1d36f7cd83a1>:107: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-8-1d36f7cd83a1>:109: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-8-1d36f7cd83a1>:109: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]


TypeError: 'module' object is not callable. Did you mean: 'random.random(...)'?

In [9]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random.random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])
            

""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.616s to generate 18804 rules
Screening rules using information gain
	Took 0.484s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!


<ipython-input-9-1e475c4824ea>:107: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-9-1e475c4824ea>:107: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-9-1e475c4824ea>:109: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-9-1e475c4824ea>:109: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]


TypeError: 'module' object is not callable. Did you mean: 'random.random(...)'?

In [10]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random.random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random.random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])
            

""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.736s to generate 18980 rules
Screening rules using information gain
	Took 0.521s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!
found a more accurate model: accuracy = 0.6388059701492538

** chain = 0, max at iter = 0 ** 
 accuracy = 0.6388059701492538, TP = 332.0,FP = 130.0, TN = 96.0, FN = 112.0
 pt_new is -726.8092999631244, prior_ChsRules=-42.45912538780067, likelihood_1 = -378.2122872472819, likelihood_2 = -306.13788732804187
 
['4_X_neg', '9_O_neg', '5_O_neg']
['5_O', '6_O_neg', '7_X_neg']
['5_O', '8_O_neg']
['3_O_neg', '7_O_neg', '5_O_neg']
[1635, 427, 1048, 1873]


<ipython-input-10-73853bcf6a16>:107: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-10-73853bcf6a16>:107: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-10-73853bcf6a16>:109: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-10-73853bcf6a16>:109: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-10-73853bcf6a16>:166: RuntimeWarning: overflow encountered in exp
  alpha = np.exp(float(pt_new -pt_curr)/T)


TypeError: 'module' object is not callable. Did you mean: 'random.random(...)'?

In [11]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random.random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random.random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])
            

""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.670s to generate 18766 rules
Screening rules using information gain
	Took 0.477s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!
found a more accurate model: accuracy = 0.6925373134328359

** chain = 0, max at iter = 0 ** 
 accuracy = 0.6925373134328359, TP = 339.0,FP = 113.0, TN = 125.0, FN = 93.0
 pt_new is -682.1402548686824, prior_ChsRules=-64.12358237369153, likelihood_1 = -344.13170495976647, likelihood_2 = -273.8849675352244
 
['5_X', '9_B', '1_O_neg']
['8_B_neg', '1_O_neg', '5_O_neg']
['8_O', '7_O_neg', '5_X']
['5_O_neg', '7_B_neg', '9_O_neg']
['9_O_neg', '3_O_neg', '8_X_neg']
['5_O_neg', '1_O_neg']
[1842, 729, 1052, 1585, 345, 1801]


<ipython-input-11-73853bcf6a16>:107: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-11-73853bcf6a16>:107: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-11-73853bcf6a16>:109: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-11-73853bcf6a16>:109: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-11-73853bcf6a16>:166: RuntimeWarning: overflow encountered in exp
  alpha = np.exp(float(pt_new -pt_curr)/T)


TypeError: 'module' object is not callable. Did you mean: 'random.random(...)'?

In [12]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random.random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random.random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random.random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])
            

""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.692s to generate 19244 rules
Screening rules using information gain
	Took 0.516s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!
found a more accurate model: accuracy = 0.5880597014925373

** chain = 0, max at iter = 0 ** 
 accuracy = 0.5880597014925373, TP = 304.0,FP = 148.0, TN = 90.0, FN = 128.0
 pt_new is -797.7256136755959, prior_ChsRules=-55.76454119002847, likelihood_1 = -408.4700343922632, likelihood_2 = -333.49103809330427
 
['4_B_neg', '1_X', '5_O_neg']
['5_X_neg', '5_B_neg', '8_X']
['5_O', '2_B_neg', '8_X_neg']
['9_B_neg', '5_O', '2_B_neg']
['7_O_neg', '3_O_neg', '5_O_neg']
[944, 1029, 369, 1599, 1851]


<ipython-input-12-c05f40bfe04e>:107: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-12-c05f40bfe04e>:107: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-12-c05f40bfe04e>:109: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-12-c05f40bfe04e>:109: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-12-c05f40bfe04e>:166: RuntimeWarning: overflow encountered in exp
  alpha = np.exp(float(pt_new -pt_curr)/T)


TypeError: 'module' object is not callable. Did you mean: 'random.random(...)'?

In [13]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier


class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random.random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random.random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random.random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random.random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])
            

""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.695s to generate 19320 rules
Screening rules using information gain
	Took 0.483s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!
found a more accurate model: accuracy = 0.5044776119402985

** chain = 0, max at iter = 0 ** 
 accuracy = 0.5044776119402985, TP = 231.0,FP = 124.0, TN = 107.0, FN = 208.0
 pt_new is -844.8555910506775, prior_ChsRules=-34.1000842041376, likelihood_1 = -351.08898420397963, likelihood_2 = -459.66652264256027
 
['5_B_neg', '5_X_neg', '2_B_neg']
['5_X_neg', '2_B_neg', '3_X_neg']
['7_O_neg', '5_O_neg', '3_O_neg']
[1789, 1707, 1833]


<ipython-input-13-bc04e0ae21e6>:107: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-13-bc04e0ae21e6>:107: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-13-bc04e0ae21e6>:109: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-13-bc04e0ae21e6>:109: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-13-bc04e0ae21e6>:166: RuntimeWarning: overflow encountered in exp
  alpha = np.exp(float(pt_new -pt_curr)/T)


NameError: name 'bisect_left' is not defined

In [14]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier
from bisect import bisect_left



class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random.random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random.random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random.random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random.random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])
            

""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.652s to generate 19180 rules
Screening rules using information gain
	Took 0.547s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!
found a more accurate model: accuracy = 0.6373134328358209

** chain = 0, max at iter = 0 ** 
 accuracy = 0.6373134328358209, TP = 222.0,FP = 26.0, TN = 205.0, FN = 217.0
 pt_new is -633.8454180328308, prior_ChsRules=-23.150756822809853, likelihood_1 = -110.71867743136954, likelihood_2 = -499.9759837786514
 
['6_X_neg', '8_B_neg', '5_X']
['5_O_neg', '1_O_neg', '9_O_neg']
[286, 1960]
found a more accurate model: accuracy = 0.7373134328358208

** chain = 0, max at iter = 1 ** 
 accuracy = 0.7373134328358208, TP = 289.0,FP = 26.0, TN = 205.0, FN = 150.0
 pt_new is -541.2566532451183, prior_ChsRules=-34.1000842041376, likelihood_1 = -113.07434913208999, likelihood_2 = -394.0822199088907
 
['6_X_neg', '8_B_neg', '5_X']
['5_O_ne

<ipython-input-14-e541f5b7b1d1>:109: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-14-e541f5b7b1d1>:109: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-14-e541f5b7b1d1>:111: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-14-e541f5b7b1d1>:111: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-14-e541f5b7b1d1>:168: RuntimeWarning: overflow encountered in exp
  alpha = np.exp(float(pt_new -pt_curr)/T)


TypeError: 'module' object is not callable. Did you mean: 'random.random(...)'?

In [15]:
import pandas as pd 
# from fim import fpgrowth,fim --> this package is difficult to install. So the rule miner can be replaced by a random forest
import numpy as np
import math
from itertools import chain, combinations
import itertools
from numpy.random import random
from random import sample
import time
import operator
from collections import Counter, defaultdict
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier
from bisect import bisect_left



class BOA(object):
    def __init__(self, binary_data,Y):
        self.df = binary_data  
        self.Y = Y
        self.attributeLevelNum = defaultdict(int) 
        self.attributeNames = []
        for i,name in enumerate(binary_data.columns):
          attribute = name.split('_')[0]
          self.attributeLevelNum[attribute] += 1
          self.attributeNames.append(attribute)
        self.attributeNames = list(set(self.attributeNames))
        

    def getPatternSpace(self):
        print('Computing sizes for pattern space ...')
        start_time = time.time()
        """ compute the rule space from the levels in each attribute """
        for item in self.attributeNames:
            self.attributeLevelNum[item+'_neg'] = self.attributeLevelNum[item]
        self.patternSpace = np.zeros(self.maxlen+1)
        tmp = [ item + '_neg' for item in self.attributeNames]
        self.attributeNames.extend(tmp)
        for k in range(1,self.maxlen+1,1):
            for subset in combinations(self.attributeNames,k):
                tmp = 1
                for i in subset:
                    tmp = tmp * self.attributeLevelNum[i]
                self.patternSpace[k] = self.patternSpace[k] + tmp
        print('\tTook %0.3fs to compute patternspace' % (time.time() - start_time))               

# This function generates rules that satisfy supp and maxlen using fpgrowth, then it selects the top N rules that make data have the biggest decrease in entropy
# there are two ways to generate rules. fpgrowth can handle cases where the maxlen is small. If maxlen<=3, fpgrowth can generates rules much faster than randomforest. 
# If maxlen is big, fpgrowh tends to generate too many rules that overflow the memories. 
    def generate_rules(self,supp,maxlen,N, method = 'randomforest'):
        self.maxlen = maxlen
        self.supp = supp
        df = 1-self.df #df has negative associations
        df.columns = [name.strip() + '_neg' for name in self.df.columns]
        df = pd.concat([self.df,df],axis = 1)
#         if method =='fpgrowth' and maxlen<=3:
#             itemMatrix = [[item for item in df.columns if row[item] ==1] for i,row in df.iterrows() ]  
#             pindex = np.where(self.Y==1)[0]
#             nindex = np.where(self.Y!=1)[0]
#             print('Generating rules using fpgrowth')
#             start_time = time.time()
#             rules= fpgrowth([itemMatrix[i] for i in pindex],supp = supp,zmin = 1,zmax = maxlen)
#             rules = [tuple(np.sort(rule[0])) for rule in rules]
#             rules = list(set(rules))
#             start_time = time.time()
#             print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
#         else:
        rules = []
        start_time = time.time()
        for length in range(1,maxlen+1,1):
            n_estimators = min(pow(df.shape[1],length),4000)
            clf = RandomForestClassifier(n_estimators = n_estimators,max_depth = length)
            clf.fit(self.df,self.Y)
            for n in range(n_estimators):
                rules.extend(extract_rules(clf.estimators_[n],df.columns))
        rules = [list(x) for x in set(tuple(x) for x in rules)]
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(rules)))
        self.screen_rules(rules,df,N) # select the top N rules using secondary criteria, information gain
        self.getPatternSpace()

    def screen_rules(self,rules,df,N):
        print('Screening rules using information gain')
        start_time = time.time()
        itemInd = {}
        for i,name in enumerate(df.columns):
            itemInd[name] = i
        indices = np.array(list(itertools.chain.from_iterable([[itemInd[x] for x in rule] for rule in rules])))
        len_rules = [len(rule) for rule in rules]
        indptr =list(accumulate(len_rules))
        indptr.insert(0,0)
        indptr = np.array(indptr)
        data = np.ones(len(indices))
        ruleMatrix = csc_matrix((data,indices,indptr),shape = (len(df.columns),len(rules)))
        mat = np.matrix(df) * ruleMatrix
        lenMatrix = np.matrix([len_rules for i in range(df.shape[0])])
        Z =  (mat ==lenMatrix).astype(int)
        Zpos = [Z[i] for i in np.where(self.Y>0)][0]
        TP = np.array(np.sum(Zpos,axis=0).tolist()[0])
        supp_select = np.where(TP>=self.supp*sum(self.Y)/100)[0]
        FP = np.array(np.sum(Z,axis = 0))[0] - TP
        TN = len(self.Y) - np.sum(self.Y) - FP
        FN = np.sum(self.Y) - TP
        p1 = TP.astype(float)/(TP+FP)
        p2 = FN.astype(float)/(FN+TN)
        pp = (TP+FP).astype(float)/(TP+FP+TN+FN)
        tpr = TP.astype(float)/(TP+FN)
        fpr = FP.astype(float)/(FP+TN)
        cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
        cond_entropy[p1*(1-p1)==0] = -((1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2)))[p1*(1-p1)==0]
        cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
        cond_entropy[p1*(1-p1)*p2*(1-p2)==0] = 0
        select = np.argsort(cond_entropy[supp_select])[::-1][-N:]
        self.rules = [rules[i] for i in supp_select[select]]
        self.RMatrix = np.array(Z[:,supp_select[select]])
        print('\tTook %0.3fs to generate %d rules' % (time.time() - start_time, len(self.rules)))

    def set_parameters(self, a1=100,b1=10,a2=100,b2=10,al=None,bl=None):
        # input al and bl are lists
        # a1/(a1 + b1) and a2/(a2 + b2) should be a  bigger than 0.5. They correspond to alpha_+, beta_+ and alpha_- and beta_- from the paper. In fact, these four values play an important role and it needs to be tuned while following the a1/(a1 + b1) >0.5 and a2/(a2 + b2) >0.5 principle
        self.alpha_1 = a1
        self.beta_1 = b1
        self.alpha_2 = a2
        self.beta_2 = b2
        if al ==None or bl==None or len(al)!=self.maxlen or len(bl)!=self.maxlen:
            print('No or wrong input for alpha_l and beta_l. The model will use default parameters!')
            self.C = [1.0/self.maxlen for i in range(self.maxlen)]
            self.C.insert(0,-1)
            self.alpha_l = [10 for i in range(self.maxlen+1)]
            self.beta_l= [10*self.patternSpace[i]/self.C[i] for i in range(self.maxlen+1)]
        else:
            self.alpha_l=[1] + list(al)
            self.beta_l = [1] + list(bl)

    def fit(self, Niteration = 300, Nchain = 1, q = 0.1, init = [], keep_most_accurate_model = True,print_message=True):
        # print('Searching for an optimal solution...')
        start_time = time.time()
        nRules = len(self.rules)
        self.rules_len = [len(rule) for rule in self.rules]
        maps = defaultdict(list)
        T0 = 1000
        split = 0.7*Niteration
        acc = {}
        most_accurate_model = defaultdict(list)
        for chain in range(Nchain):
            # initialize with a random pattern set
            if init !=[]:
                rules_curr = init.copy()
            else:
                N = sample(range(1,min(8,nRules),1),1)[0]
                rules_curr = sample(range(nRules),N)
            rules_curr_norm = self.normalize(rules_curr)
            pt_curr = -100000000000
            maps[chain].append([-1,[pt_curr/3,pt_curr/3,pt_curr/3],rules_curr,[self.rules[i] for i in rules_curr]])
            acc[chain] = 0
            for iter in range(Niteration):
                if iter>=split:
                    p = np.array(range(1+len(maps[chain])))
                    p = np.array(list(accumulate(p)))
                    p = p/p[-1]
                    index = find_lt(p,random.random())
                    rules_curr = maps[chain][index][2].copy()
                    rules_curr_norm = maps[chain][index][2].copy()
                rules_new, rules_norm = self.propose(rules_curr.copy(), rules_curr_norm.copy(),q)
                cfmatrix,prob =  self.compute_prob(rules_new)
                T = T0**(1 - iter/Niteration)
                pt_new = sum(prob)
                alpha = np.exp(float(pt_new -pt_curr)/T)
                
                if (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)> acc[chain]:
                    most_accurate_model[chain] = rules_new[:]
                    acc[chain] = (cfmatrix[0] + cfmatrix[2])/sum(cfmatrix)
                    if print_message:
                        print('found a more accurate model: accuracy = {}'.format(acc[chain]))
                        
                if pt_new > sum(maps[chain][-1][1]):
                    maps[chain].append([iter,prob,rules_new,[self.rules[i] for i in rules_new]])
                    if print_message:
                        print('\n** chain = {}, max at iter = {} ** \n accuracy = {}, TP = {},FP = {}, TN = {}, FN = {}\n pt_new is {}, prior_ChsRules={}, likelihood_1 = {}, likelihood_2 = {}\n '.format(chain, iter,(cfmatrix[0]+cfmatrix[2]+0.0)/len(self.Y),cfmatrix[0],cfmatrix[1],cfmatrix[2],cfmatrix[3],sum(prob), prob[0], prob[1], prob[2]))
                        # print '\n** chain = {}, max at iter = {} ** \n obj = {}, prior = {}, llh = {} '.format(chain, iter,prior+llh,prior,llh)
                        self.print_rules(rules_new)
                        print(rules_new)
                if random.random() <= alpha:
                    rules_curr_norm,rules_curr,pt_curr = rules_norm.copy(),rules_new.copy(),pt_new
        # print('\tTook %0.3fs to generate an optimal rule set' % (time.time() - start_time))
        if keep_most_accurate_model:
            acc_list = [acc[chain] for chain in range(Nchain)]
            index = acc_list.index(max(acc_list))
            return [self.rules[i] for i in most_accurate_model[index]] 
        else:
            pt_max = [sum(maps[chain][-1][1]) for chain in range(Nchain)]
            index = pt_max.index(max(pt_max))
            return maps[index][-1][3]

    def propose(self, rules_curr,rules_norm,q):
        nRules = len(self.rules)
        Yhat = (np.sum(self.RMatrix[:,rules_curr],axis = 1)>0).astype(int)
        incorr = np.where(self.Y!=Yhat)[0]
        N = len(rules_curr)
        if len(incorr)==0:
            clean = True
            move = ['clean']
            # it means the BOA correctly classified all points but there could be redundant patterns, so cleaning is needed
        else:
            clean = False
            ex = sample(incorr.tolist(),1)[0]
            t = random.random()
            if self.Y[ex]==1 or N==1:
                if t<1.0/2 or N==1:
                    move = ['add']       # action: add
                else:
                    move = ['cut','add'] # action: replace
            else:
                if t<1.0/2:
                    move = ['cut']       # action: cut
                else:
                    move = ['cut','add'] # action: replace
        if move[0]=='cut':
            """ cut """
            if random.random()<q:
                candidate = list(set(np.where(self.RMatrix[ex,:]==1)[0]).intersection(rules_curr))
                if len(candidate)==0:
                    candidate = rules_curr
                cut_rule = sample(candidate,1)[0]
            else:
                p = []
                all_sum = np.sum(self.RMatrix[:,rules_curr],axis = 1)
                for index,rule in enumerate(rules_curr):
                    Yhat= ((all_sum - np.array(self.RMatrix[:,rule]))>0).astype(int)
                    TP,FP,TN,FN  = getConfusion(Yhat,self.Y)
                    p.append(TP.astype(float)/(TP+FP+1))
                    # p.append(log_betabin(TP,TP+FP,self.alpha_1,self.beta_1) + log_betabin(FN,FN+TN,self.alpha_2,self.beta_2))
                p = [x - min(p) for x in p]
                p = np.exp(p)
                p = np.insert(p,0,0)
                p = np.array(list(accumulate(p)))
                if p[-1]==0:
                    index = sample(range(len(rules_curr)),1)[0]
                else:
                    p = p/p[-1]
                index = find_lt(p,random.random())
                cut_rule = rules_curr[index]
            rules_curr.remove(cut_rule)
            rules_norm = self.normalize(rules_curr)
            move.remove('cut')
            
        if len(move)>0 and move[0]=='add':
            """ add """
            if random.random()<q:
                add_rule = sample(range(nRules),1)[0]
            else: 
                Yhat_neg_index = list(np.where(np.sum(self.RMatrix[:,rules_curr],axis = 1)<1)[0])
                mat = np.multiply(self.RMatrix[Yhat_neg_index,:].transpose(),self.Y[Yhat_neg_index])
                # TP = np.array(np.sum(mat,axis = 0).tolist()[0])
                TP = np.sum(mat,axis = 1)
                FP = np.array((np.sum(self.RMatrix[Yhat_neg_index,:],axis = 0) - TP))
                TN = np.sum(self.Y[Yhat_neg_index]==0)-FP
                FN = sum(self.Y[Yhat_neg_index]) - TP
                p = (TP.astype(float)/(TP+FP+1))
                p[rules_curr]=0
                add_rule = sample(np.where(p==max(p))[0].tolist(),1)[0]
            if add_rule not in rules_curr:
                rules_curr.append(add_rule)
                rules_norm = self.normalize(rules_curr)

        if len(move)>0 and move[0]=='clean':
            remove = []
            for i,rule in enumerate(rules_norm):
                Yhat = (np.sum(self.RMatrix[:,[rule for j,rule in enumerate(rules_norm) if (j!=i and j not in remove)]],axis = 1)>0).astype(int)
                TP,FP,TN,FN = getConfusion(Yhat,self.Y)
                if TP+FP==0:
                    remove.append(i)
            for x in remove:
                rules_norm.remove(x)
            return rules_curr, rules_norm
        return rules_curr, rules_norm

    def compute_prob(self,rules):
        Yhat = (np.sum(self.RMatrix[:,rules],axis = 1)>0).astype(int)
        TP,FP,TN,FN = getConfusion(Yhat,self.Y)
        Kn_count = list(np.bincount([self.rules_len[x] for x in rules], minlength = self.maxlen+1))
        prior_ChsRules= sum([log_betabin(Kn_count[i],self.patternSpace[i],self.alpha_l[i],self.beta_l[i]) for i in range(1,len(Kn_count),1)])            
        likelihood_1 =  log_betabin(TP,TP+FP,self.alpha_1,self.beta_1)
        likelihood_2 = log_betabin(TN,FN+TN,self.alpha_2,self.beta_2)
        return [TP,FP,TN,FN],[prior_ChsRules,likelihood_1,likelihood_2]

    def normalize_add(self, rules_new, rule_index):
        rules = rules_new.copy()
        for rule in rules_new:
            if set(self.rules[rule]).issubset(self.rules[rule_index]):
                return rules_new.copy()
            if set(self.rules[rule_index]).issubset(self.rules[rule]):
                rules.remove(rule)
        rules.append(rule_index)
        return rules

    def normalize(self, rules_new):
        try:
            rules_len = [len(self.rules[index]) for index in rules_new]
            rules = [rules_new[i] for i in np.argsort(rules_len)[::-1][:len(rules_len)]]
            p1 = 0
            while p1<len(rules):
                for p2 in range(p1+1,len(rules),1):
                    if set(self.rules[rules[p2]]).issubset(set(self.rules[rules[p1]])):
                        rules.remove(rules[p1])
                        p1 -= 1
                        break
                p1 += 1
            return rules
        except:
            return rules_new.copy()


    def print_rules(self, rules_max):
        for rule_index in rules_max:
            print(self.rules[rule_index])
            

""" parameters """
# The following parameters are recommended to change depending on the size and complexity of the data
N = 2000      # number of rules to be used in SA_patternbased and also the output of generate_rules
Niteration = 500  # number of iterations in each chain
Nchain = 2         # number of chains in the simulated annealing search algorithm

supp = 5           # 5% is a generally good number. The higher this supp, the 'larger' a pattern is
maxlen = 3         # maxmum length of a pattern

# \rho = alpha/(alpha+beta). Make sure \rho is close to one when choosing alpha and beta. 
alpha_1 = 500       # alpha_+
beta_1 = 1          # beta_+
alpha_2 = 500         # alpha_-
beta_2 = 1       # beta_-

""" input file """
# notice that in the example, X is already binary coded. 
# Data has to be binary coded and the column name shd have the form: attributename_attributevalue
filepathX = 'tictactoe_X.txt' # input file X
filepathY = 'tictactoe_Y.txt' # input file Y
df = pd.read_csv(filepathX,header=0,sep=" ")
Y = np.loadtxt(open(filepathY,"rb"),delimiter=" ")

import random
lenY = len(Y)
train_index = random.sample(range(lenY), int(0.70 * lenY))
test_index = [i for i in range(lenY) if i not in train_index]

model = BOA(df.iloc[train_index],Y[train_index])
model.generate_rules(supp,maxlen,N)
model.set_parameters(alpha_1,beta_1,alpha_2,beta_2,None,None)
rules = model.fit(Niteration,Nchain,print_message=True)

# test
Yhat = predict(rules,df.iloc[test_index])
TP,FP,TN,FN = getConfusion(Yhat,Y[test_index])
tpr = float(TP)/(TP+FN)
fpr = float(FP)/(FP+TN)
print('TP = {}, FP = {}, TN = {}, FN = {} \n accuracy = {}, tpr = {}, fpr = {}'.format(TP,FP,TN,FN, float(TP+TN)/(TP+TN+FP+FN),tpr,fpr))

	Took 3.792s to generate 18745 rules
Screening rules using information gain


<ipython-input-15-4469ac129a39>:109: RuntimeWarning: divide by zero encountered in log
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-15-4469ac129a39>:109: RuntimeWarning: invalid value encountered in multiply
  cond_entropy = -pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1))-(1-pp)*(p2*np.log(p2)+(1-p2)*np.log(1-p2))
<ipython-input-15-4469ac129a39>:111: RuntimeWarning: divide by zero encountered in log
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-15-4469ac129a39>:111: RuntimeWarning: invalid value encountered in multiply
  cond_entropy[p2*(1-p2)==0] = -(pp*(p1*np.log(p1)+(1-p1)*np.log(1-p1)))[p2*(1-p2)==0]
<ipython-input-15-4469ac129a39>:168: RuntimeWarning: overflow encountered in exp
  alpha = np.exp(float(pt_new -pt_curr)/T)


	Took 0.481s to generate 2000 rules
Computing sizes for pattern space ...
	Took 0.000s to compute patternspace
No or wrong input for alpha_l and beta_l. The model will use default parameters!
found a more accurate model: accuracy = 0.6537313432835821

** chain = 0, max at iter = 0 ** 
 accuracy = 0.6537313432835821, TP = 348.0,FP = 149.0, TN = 90.0, FN = 83.0
 pt_new is -732.6346666948652, prior_ChsRules=-66.49072063146559, likelihood_1 = -417.6734608631941, likelihood_2 = -248.47048520020553
 
['4_X_neg', '3_X', '5_O_neg']
['1_O', '8_X', '5_X_neg']
['5_X', '9_O_neg', '7_B_neg']
['6_O_neg', '5_O', '2_B_neg']
['1_X', '7_O_neg', '5_O_neg']
['5_X', '6_X', '4_X']
[775, 491, 878, 975, 803, 1881]
found a more accurate model: accuracy = 0.664179104477612

** chain = 0, max at iter = 1 ** 
 accuracy = 0.664179104477612, TP = 344.0,FP = 138.0, TN = 101.0, FN = 87.0
 pt_new is -720.5796264639825, prior_ChsRules=-66.49072063146559, likelihood_1 = -395.8330315016501, likelihood_2 = -258.2558743308